# Proyecto Final

_Jorge III Altamirano Astorga - 175904_

In [14]:
from bs4 import BeautifulSoup
import bs4 as bs4
import requests
import sys
from dateutil import parser #esta lib es más útil que la de Python para interpretar fechas parciales, como las de Twitter
import re

In [2]:
NEWSPPR="elfinanciero_mx"
STOCK="bimbo"
YEAR="2018"
q = 'https://mobile.twitter.com/search?q=' + STOCK + \
      '%20from%3A' + NEWSPPR + \
      '%20since%3A'+YEAR+'-01-01%20until%3A'+YEAR+'-12-31&src=typd&lang=en'
q

'https://mobile.twitter.com/search?q=bimbo%20from%3Aelfinanciero_mx%20since%3A2018-01-01%20until%3A2018-12-31&src=typd&lang=en'

In [56]:
def extract_tweets(soup, file, i, year):
    tweets = soup.find_all("table", attrs={"class": "tweet"})
    # soup.find_all("div", attrs={"class": "tweet-text"})
    # tweet = tweets[0]
    for tweet in tweets:
        tweet_text = tweet.find_all('div', attrs={'class': 'tweet-text'})
        tweet_date = tweet.find_all("td", attrs={'class': 'timestamp'})
        for text in tweet_text:
            if type(text) == bs4.element.Tag:
                txt = re.sub("(eleconomista\.com\.mx|pic\.twitter\.com|bit\.ly|bitly\.com)/[^ ]*", " ", text.text).strip()
                txt = re.sub("(\||\n)", " ", txt)
                file.write(txt + "|")
        for date in tweet_date:
            if type(date) == bs4.element.Tag:
                date = date.text.strip() 
                date = re.sub(year[-2:] + "$", "", date)
                file.write(str(parser.parse(date + " " + year) )  + "|%s"%i + "\n")
                
# get_tweets(year="2019", filename="bmv.eleconomista")

In [53]:
def get_tweets(query="bolsa OR IPC OR BMV", newspaper="elfinanciero_mx", year="2018", filename="file"):
    q = 'https://mobile.twitter.com/search?q=' + query + \
          '%20from%3A' + newspaper + \
          '%20since%3A'+year+'-01-01%20until%3A'+year+'-12-31&src=typd&lang=en'

    with open("%s.%s.csv"%(filename,year), "wt") as file:
        i = 0
        q = 'https://mobile.twitter.com/search?q=' + query + \
        '%20from%3A' + newspaper + '%20since%3A' + year + \
        '-01-01%20until%3A' + year +'-12-31&src=typd&lang=en'
        file.write("tweet|time|page\n") #header
        while(True): #loop para extraer todas las páginas 
            print("  Downloading page %d..."%(i+1))
            try:
                response = requests.get(q)
            except:
                raise Exception("Error al obtener página de resultados.")
            if response.status_code != 200:
                raise Exception("Error (!= 200) al obtener página de resultados.")
            soup = BeautifulSoup(response.text, 'lxml') #abre la primera página de búsqueda de twitter
            extract_tweets(soup, file, i, year)

            #with open("%s_%d.html"%(query,i), 'w') as html:
            #    html.write(response.text)

            nx = soup.find("a", text=" Load older Tweets ")
            if nx != None and nx.has_attr('href'):
                q = "https://mobile.twitter.com" + nx.get_attribute_list('href')[0]
            else:
                break
            i += 1
    print("  Done!")

In [51]:
for yr in ["2017", "2018", "2019"]:
    for source in ["elfinanciero_mx", "eleconomista"]:
        print("Downloading year %s from @%s"%(yr,source))
        get_tweets(query="bolsa OR IPC OR BMV", newspaper=source, year=yr, filename="bmv.%s"%source)

Done!
Done!
Done!
Done!
Done!
Done!
